In [2]:
#importar librerías.
import mysql.connector
import pandas as pd
import numpy as np
import locale
from faker import Faker
from datetime import datetime, timedelta

# Configuraciones generales.
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')
fake = Faker(['es_ES'])
Faker.seed(12345)
np.random.seed(12345)

## Generar Data.

In [3]:
# dim_fecha
start_date = datetime(2020, 1, 1)
end_date = datetime.now()
date_range = pd.date_range(start=start_date, end=end_date)
dim_fecha = pd.DataFrame({
    'id_fecha': range(1, len(date_range) + 1),
    'fecha': date_range,
    'año': date_range.year,
    'mes': date_range.month,
    'dia': date_range.day,
    'dia_semana': date_range.strftime('%A'),
    'semana': date_range.isocalendar().week.astype(int),
    'trimestre': (date_range.month - 1) // 3 + 1
})

# dim_merchant
num_merchants = len(date_range)
dim_merchant = pd.DataFrame({
    'id_merchant': range(1, num_merchants + 1),
    'nombre_merchant': [f'Merchant {fake.company()}' for _ in range(num_merchants)],
    'email_merchant': [fake.company_email() for _ in range(num_merchants)],
    'telefono_merchant': [fake.phone_number() for _ in range(num_merchants)]
})

# dim_remitente
num_personas = len(date_range)
dim_remitente = pd.DataFrame({
    'id_remitente': range(1, num_personas + 1),
    'tipo_documento': np.random.choice(['DNI', 'Pasaporte'], num_personas),
    'numero_documento': [fake.unique.random_number(digits=8) for _ in range(num_personas)],
    'nombres': [fake.name() for _ in range(num_personas)],
    'email': [fake.email() for _ in range(num_personas)],
    'telefono': [fake.phone_number() for _ in range(num_personas)]
})

# dim_beneficiario
dim_beneficiario = pd.DataFrame({
    'id_beneficiario': range(1, num_personas + 1),
    'tipo_documento': np.random.choice(['DNI', 'Pasaporte'], num_personas),
    'numero_documento': [fake.unique.random_number(digits=8) for _ in range(num_personas)],
    'nombres': [fake.name() for _ in range(num_personas)],
    'email': [fake.email() for _ in range(num_personas)],
    'telefono': [fake.phone_number() for _ in range(num_personas)]
})

# dim_moneda
dim_moneda = pd.DataFrame({
    'id_moneda': [1, 2],
    'codigo_moneda': ['USD', 'EUR'],
    'descripcion_moneda': ['Dólar Estadounidense', 'Euro']
})

# fact_remesas
num_remesas = len(date_range)
fact_remesas = pd.DataFrame({
    'id_remesa': range(1, num_remesas + 1),
    'id_merchant': np.random.choice(dim_merchant['id_merchant'], num_remesas),
    'id_remitente': np.random.choice(dim_remitente['id_remitente'], num_remesas),
    'id_beneficiario': np.random.choice(dim_beneficiario['id_beneficiario'], num_remesas),
    'id_moneda': np.random.choice(dim_moneda['id_moneda'], num_remesas),
    'id_fecha': np.random.choice(dim_fecha['id_fecha'], num_remesas),
    'monto': np.random.uniform(10, 1000, num_remesas).round(2),
    'estado_remesa': np.random.choice(['Pendiente', 'Entregado', 'Cancelado'], num_remesas)
})

In [5]:
connection = mysql.connector.connect(
    host='localhost',
    database='dm_remesas',
    user='root',
    password=''
)
cursor = connection.cursor()

## Cargar Datos a MySQL.

In [ ]:
# dim_fecha
insert_query = """
INSERT INTO dim_fecha (id_fecha, fecha, año, mes, dia, dia_semana, semana, trimestre) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
data_list = list(dim_fecha.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()

# dim_merchant
insert_query = """
INSERT INTO dim_merchant (id_merchant, nombre_merchant, email_merchant, telefono_merchant) 
VALUES (%s, %s, %s, %s)
"""
data_list = list(dim_merchant.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()

# dim_remitente
insert_query = """
INSERT INTO dim_remitente (id_remitente, tipo_documento, numero_documento, nombres, email, telefono) 
VALUES (%s, %s, %s, %s, %s, %s)
"""
data_list = list(dim_remitente.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()

# dim_beneficiario
insert_query = """
INSERT INTO dim_beneficiario (id_beneficiario, tipo_documento, numero_documento, nombres, email, telefono) 
VALUES (%s, %s, %s, %s, %s, %s)
"""
data_list = list(dim_beneficiario.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()

# dim_moneda
insert_query = """
INSERT INTO dim_moneda (id_moneda, codigo_moneda, descripcion_moneda) 
VALUES (%s, %s, %s)
"""
data_list = list(dim_moneda.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()

# fact_remesas
insert_query = """
INSERT INTO fact_remesas (id_remesa, id_merchant, id_remitente, id_beneficiario, id_moneda, id_fecha, monto, estado_remesa) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
data_list = list(fact_remesas.itertuples(index=False))
cursor.executemany(insert_query, data_list)
connection.commit()
cursor.close()
connection.close()